In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import re
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
with open("d24_input.txt", "r") as f:
    lines = f.read().split("\n")
# lines = [(i[0], int(i[1:])) for i in lines]
lines[0], lines[-1]

('seweswweswnwseeeesenwneseeseee', 'eneeneeeeeeeeseenew')

In [3]:
def replace_direction(s):
    # ne -> 1
    # se -> 2
    # sw -> 3
    # nw -> 4
    return s.replace("ne", "1").replace("se", "2").replace("sw", "3").replace("nw", "4")


def get_next_tile(x, y, l):
    if l == "e":
        return x + 2, y
    if l == "w":
        return x - 2, y
    if l == "1":
        return x + 1, y + 1
    if l == "2":
        return x + 1, y - 1
    if l == "3":
        return x - 1, y - 1
    if l == "4":
        return x - 1, y + 1
    print(f"Error; {l}")
    return None


def get_final_tile(moves, x=0, y=0):
    for l in replace_direction(moves):
        x, y = get_next_tile(x, y, l)
    return x, y

In [4]:
tile_dict = {"0,0": 0}  # 0 is white, 1 is black
for l in lines:
    x, y = get_final_tile(l)
    key = str(x) + "," + str(y)
    if key in tile_dict:
        tile_dict[key] = 1 - tile_dict[key]
    else:
        tile_dict[key] = 1

In [5]:
sum(tile_dict.values())

427

In [6]:
def check_color(x, y, tile_dict):
    key = str(x) + "," + str(y)
    if key not in tile_dict:
        tile_dict[key] = 0
    return tile_dict[key]


def check_neighbour(x, y, tile_dict):
    count = 0
    count += check_color(x + 2, y, tile_dict)
    count += check_color(x - 2, y, tile_dict)
    count += check_color(x + 1, y + 1, tile_dict)
    count += check_color(x + 1, y - 1, tile_dict)
    count += check_color(x - 1, y - 1, tile_dict)
    count += check_color(x - 1, y + 1, tile_dict)
    return count

In [7]:
init_tile_dict = tile_dict.copy()

tile_dict = init_tile_dict.copy()

for _ in tqdm(range(100)):
    # use check_neighbour to expand
    keys_to_check = list(tile_dict.keys())
    for k in keys_to_check:
        x, y = int(k.split(",")[0]), int(k.split(",")[1])
        check_neighbour(x, y, tile_dict)

    change_key_list = []
    keys_to_check = list(tile_dict.keys())
    for k in keys_to_check:
        x, y = int(k.split(",")[0]), int(k.split(",")[1])
        black_num = check_neighbour(x, y, tile_dict)
        color = tile_dict[k]
        if ((color == 1) and ((black_num == 0) or (black_num > 2))) or (
            (color == 0) and (black_num == 2)
        ):
            change_key_list.append(k)
    new_dict = tile_dict.copy()
    for k in change_key_list:
        new_dict[k] = 1 - new_dict[k]
    tile_dict = new_dict.copy()

100%|████████████████████████████████████████████████████████████████| 100/100 [01:04<00:00,  1.56it/s]


In [8]:
sum(tile_dict.values())

3837